# Sprite/animation development examples

This Jupyter notebook contains examples of sprite and animation code that can be used as a template to add scenes to the display montage.

For those unfamiliar with Jupyter, simply click on a cell containing code and press **CONTROL + ENTER** at the same time to run it. If you accidentally double-click on a Mardown cell (like this one), you can press the same **CONTROL + ENTER** to return it to HTML formatting. 

If something is taking too long to run, you can kill it with the stop button above or by going to **kernel** > **interrupt**.

You can save your work via **File** > **Save and Checkpoint**. When you relaunch the notebook, it'll be in the same state. If you relaunch Jupyter (e.g., because you turned off your Raspberry Pi), you'll have to rerun the import cell.

Finally, don't worry about warning messages such as:

```
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database',)).History will not be written to the database.
```

As far as I can tell, they're a result of running Jupyter as root, which is required on Raspberry Pi. If someone has a solution, hit me up on twitter: [@DocBrown](https://twitter.com/DocBrownPhD)

#### Don't forget to run the cell containing the imports first!

In [ ]:
#Imports - run this cell first!

import sys
import re
import time

from collections import namedtuple, deque
from itertools import cycle, chain, repeat

import numpy as np

from PIL import Image
import rgbmatrix as rgb

sys.path.append("/home/pi/8bit_raspi_pixel_art_display/")
from settings import (NES_PALETTE_HEX, dispmatrix)
from core import *

#The following lines import the data from the individual game 
#animation routines so that you can look at the sprites.
from sprites.zelda2 import zelda2_animation
from sprites.finalfantasy import finalfantasy_animation
from sprites.megaman2 import megaman2_animation
from sprites.ninjagaiden import ninjagaiden_animation
from sprites.blastermaster import blastermaster_animation
from sprites.dragonwarrior import dragonwarrior_animation
from sprites.supermariobros3 import smb3_animation
from sprites.castlevania3 import castlevania3_animation
from sprites.dragonstrike import dragonstrike_animation
from sprites.excitebike import excitebike_animation
from sprites.kirbysadventure import kirbysadventure_animation
from sprites.lifeforce import lifeforce_animation
from sprites.ducktales import ducktales_animation

### Clearing the display

Most of the functions have the option to clear the display after running. If, for some reason, while playing with the code, an image becomes frozen on the RGB LED matrix, `dispmatrix.Clear()` will clear it. Otherwise, if you try to display another sprite, they'll both display at the same time and blend together.

In [ ]:
dispmatrix.Clear() #Clear the RGB LED matrix

## Sprites 

Sprites are defined via sprite class, which is defined in `core.py`. They are intialized via two variables: `palette`, a dictionary that defines the colors the sprite will use and assigns them to letters, and `matrix`, a list of strings that define the the positions of individual pixels from top to bottom.

### palette

The NES palette is defined in a two-dimensional numpy array, `NES_PALETTE_HEX` that encodes all of the RGB values supported by the NES as represented by following image:

![NES palette](images/nes_palette.jpg)

All NES sprites are constructed from these colors. Individual colors are specified by `NES_PALETTE_HEX[ROW, COLUMN]`, so for example, white corresponds to `NES_PALETTE_HEX[0, 3]`, while bright green would be `NES_PALETTE_HEX[1, 9]`.

The dictionary passed to palette follows the following format:

```
palette = { 
        "b":NES_PALETTE_HEX[0, 13], #Black
        "w":NES_PALETTE_HEX[3, 0],  #White
        "r":NES_PALETTE_HEX[1, 6],  #Red
        "y":NES_PALETTE_HEX[2, 8],  #Yellow
        "l":NES_PALETTE_HEX[3, 1],  #Light blue
    },
```

Each color is assigned to a string (preferably a single letter) that will be used to define the matrix. There are two important restrictions to these strings that will become apparent below:

1. The string must contain only letters, it CANNOT contain any digits (i.e., numbers).
2. The string cannot be the letter 'x' - this letter is reserved to define transparent pixels (see below).

### matrix

The matrix defines the position of the pixels as a list of pixel rows, from top to bottom. For example, given the palette defined above, we could define a 5 x 5 sprite as follows:

```
matrix = [
        "x2y1x2",  #transparent, transparent, yellow, transparent, transparent  
        "x1y3x1", #transparent, yellow, yellow, yellow, transparent
        "y5",     #yellow, yellow, yellow, yellow, yellow,
        "x1y3x1", #transparent, yellow, yellow, yellow, transparent
        "x2y1x2",  #transparent, transparent, yellow, transparent, transparent
         ]
```        

As you can see, the syntax for each row is "COLOR1|LENGTH1|COLOR2|LENGTH2", with no spaces (the | are there for illustrative purposes). The letter `x` defines transparent sprites - when these positions are drawn over a background sprite (explained below) the background is not obscured. As was the case with the palette, there are a few restrictions on the matrix:

1. Matrices are rectangles: all rows must be the same length as verified by the sum of the numbers in each row.
2. The maximum dimensions of the matrix should be no larger than 32 rows and 32 columns. While the code may accept a larger matrix, it's unlikely to be plotted correctly given that the codebase is written assuming a 32 x 32 RGB LED display.

Let's try defining and displaying the above sprite using the `display_sprite()` function: 

In [ ]:
test = sprite(
    palette = { 
        "b":NES_PALETTE_HEX[0, 13],
        "w":NES_PALETTE_HEX[3, 0],
        "r":NES_PALETTE_HEX[1, 6],
        "y":NES_PALETTE_HEX[2, 8],
        "l":NES_PALETTE_HEX[3, 1],
    },
    matrix = [
        "x2y1x2",  #transparent, transparent, yellow, transparent, transparent  
        "x1y3x1", #transparent, yellow, yellow, yellow, transparent
        "y5",     #yellow, yellow, yellow, yellow, yellow,
        "x1y3x1", #transparent, yellow, yellow, yellow, transparent
        "x2y1x2",  #transparent, transparent, yellow, transparent, transparent
         ]    
    )

#The display_sprite function displays the sprite and returns a 32 x 32 numpy array
#containing the contents of the display in per-pixel RGB values. Try playing with
#The values to see how sprites move.

_ = display_sprite(dispmatrix=dispmatrix, #dispmatrix defines the details required to run the display
                   sprite=test,           #The sprite to display in the foreground
                   bg_sprite=None,        #The sprite to display in the background (black if none)
                   center=True,           #Center the sprite? If False, sprite will be in upper right.
                   display=True,          #If false, don't display, just return numpy array.
                   clear=True,            #Clear RGB matrix before displaying?
                   xoff=0,                #offset on x-axis in pixels: positive is left, negative is right
                   yoff=0,                #offset on y-axis in pixels: positive is down, negative is up
                  )

### Background sprites

Background sprites are defined the exact same way as above. For best results, the sprite should be 32 x 32 and fill up the whole display. Also background sprites shouldn't use the transparent sprite character `x` as they are the background.

I make my backgrounds from NES tiles (see [The Spriter's Ressource](https://www.spriters-resource.com/nes/), for example), and use the `xoff` and `yoff` options to properly place the sprite relative to the background. 

The following example is Scrooge McDuck from *DuckTales* in the Amazon stage:

In [ ]:
ScroogeWalkRight01 = sprite(
    palette = { 
        "b":NES_PALETTE_HEX[0, 13],
        "w":NES_PALETTE_HEX[3, 0],
        "r":NES_PALETTE_HEX[1, 6],
        "y":NES_PALETTE_HEX[2, 8],
        "l":NES_PALETTE_HEX[3, 1],
    },
    matrix = [
        "x13b6x4",
        "x12b4w1b3x3",
        "x11b5w2b2x3",
        "x11b6w1b2x3",
        "x11b6r1b2x3",
        "x12b4r3b1x3",
        "x13b2r1b7",
        "x11b5w3b1w1b1x1",
        "x10b5w1l2w2l1w1x1",
        "x10b4w1l4w1l2x1",
        "x12b2w1l2b2w1b1l1x1",
        "x12w3l2b2w1b1l1w1",
        "x9b4w3l1b2y1b1y1w1",
        "x7b2r2b1w2b1y3w1l1y1l1x1",
        "x6b1r4b6y6",
        "x4b2r6b2w1b8",
        "x3b1w1b2r1b2r2b6x2b2x1",
        "x3b1w1b4r3b2r3b3w2b1",
        "x4b1w2b3r4b1w1r1b1r2w2b1",
        "x4b1w2b1w1b1r4b2r1b1r3b1x1",
        "x4b4w2b2r5b4x2",
        "x3b1y1b2w4b6x1b2x3",
        "x2b1y1b1x1b1w9b2y2b1x2",
        "x1b1y1b1x1b1x1b2w1y3b2w1b1y3b1x2",
        "b1y1b1x1b1y1b1y2b3y3b1y4b1x2",
        "y1b1x2b1y3b1x3b2y1b1w2y1b1x3",
        "b1x3b1w1b2x6b1y1w2b1x4",
        "x4b1w2b2x5b1y2b1x5",
        "x4b1y4b1x5b2x6",
    ]    
)

DuckTalesBG01 = sprite(
    palette = {
        "b":NES_PALETTE_HEX[0, 13],
        "g":NES_PALETTE_HEX[2, 9],
        "d":NES_PALETTE_HEX[1, 10],
        "l":NES_PALETTE_HEX[1, 1],
        "r":NES_PALETTE_HEX[0, 7],
    },
    matrix = [
        "l16" + "b2r1g2r1g2r1b1r3b3",
        "l16" + "g1b1r1g1b1g1r1g1r1b1r1b1r1b3",
        "l16" + "g1b1r1g1b1g4b1r1b1r1b3",
        "l16" + "b1g1b1g1b1g3r1g1r1b1r1b3",
        "l16" + "b1g1b1r1b1g2r1b1g1r1b1r2b2",
        "l16" + "b2g1b2g2r1b1g1r1b1r2b2",
        "l16" + "b3g1b3r1b1g1r1b1r1b3",
        "l16" + "b1r1b2g2b3g1r1b1r2b2",
        "l16" + "b1r1b4g2b4r2b2",
        "l16" + "b1r1b1g1b4g3b5",
        "l16" + "b1r1b1g2b1g1b4g3b2",
        "l16" + "b1r1b1g2b1g1r1g1r1b4g1b1",
        "l8b2l5b1" + "b1r3g1b1g3r1b1r2b2g1",
        "l1b2l4b2l2b1l2b2" + "b2r1b1r1b1g2r2g1b1r1b2g1",
        "b2l2b1l1b6l1b3" + "b1r2b1g1r1g2r1b1g1b1r2b2",
        "b16" + "b16",
        "b6d1b6d1b2" + "b6d1b6d1b2",
        "b5d2b5d2b2" + "b5d2b5d2b2",
        "b1d1b2d2b2d1b2d2b3" + "b1d1b2d2b2d1b2d2b3",
        "d2b2d2b1d2b1d3b3" + "d2b2d2b1d2b1d3b3",
        "d1b2d3b1d1b2d3b2d1" + "d1b2d3b1d1b2d3b2d1",
        "d1b2d2b1d2b1d3b2d2" + "d1b2d2b1d2b1d3b2d2",
        "d1b1d3b1d2b1d3b1d3" + "d1b1d3b1d2b1d3b1d3",
        "b2d3b1d1b1d3b1d4" + "b2d3b1d1b1d3b1d4",
        "b1d4b2d4b1d4" + "b1d4b2d4b1d4",
        "d5b2d3b1d5" + "d5b2d3b1d5",
        "d5b1d10" + "d5b1d10",
        "d5b1d9b1" + "d5b1d9b1",
        "d4b2d9b1" + "d4b2d9b1",
        "b1d3b2d8b2" + "b1d3b2d8b2",
        "b7d2b2d1b2d1b1" + "b7d2b2d1b2d1b1",
        "b1g1d1b1d2b3g2b1g2b2" + "b1g1d1b1d2b3g2b1g2b2",
    ]    
)

_ = display_sprite(dispmatrix=dispmatrix, 
                   sprite=ScroogeWalkRight01, 
                   bg_sprite=DuckTalesBG01, 
                   center=True,
                   display=True,
                   clear=True,
                   yoff=1,
                   xoff=2)

## Animations

Descriptions to be added, example code below:

In [ ]:
ScroogeWalkRight01 = sprite(
    palette = { 
        "b":NES_PALETTE_HEX[0, 13],
        "w":NES_PALETTE_HEX[3, 0],
        "r":NES_PALETTE_HEX[1, 6],
        "y":NES_PALETTE_HEX[2, 8],
        "l":NES_PALETTE_HEX[3, 1],
    },
    matrix = [
        "x13b6x4",
        "x12b4w1b3x3",
        "x11b5w2b2x3",
        "x11b6w1b2x3",
        "x11b6r1b2x3",
        "x12b4r3b1x3",
        "x13b2r1b7",
        "x11b5w3b1w1b1x1",
        "x10b5w1l2w2l1w1x1",
        "x10b4w1l4w1l2x1",
        "x12b2w1l2b2w1b1l1x1",
        "x12w3l2b2w1b1l1w1",
        "x9b4w3l1b2y1b1y1w1",
        "x7b2r2b1w2b1y3w1l1y1l1x1",
        "x6b1r4b6y6",
        "x4b2r6b2w1b8",
        "x3b1w1b2r1b2r2b6x2b2x1",
        "x3b1w1b4r3b2r3b3w2b1",
        "x4b1w2b3r4b1w1r1b1r2w2b1",
        "x4b1w2b1w1b1r4b2r1b1r3b1x1",
        "x4b4w2b2r5b4x2",
        "x3b1y1b2w4b6x1b2x3",
        "x2b1y1b1x1b1w9b2y2b1x2",
        "x1b1y1b1x1b1x1b2w1y3b2w1b1y3b1x2",
        "b1y1b1x1b1y1b1y2b3y3b1y4b1x2",
        "y1b1x2b1y3b1x3b2y1b1w2y1b1x3",
        "b1x3b1w1b2x6b1y1w2b1x4",
        "x4b1w2b2x5b1y2b1x5",
        "x4b1y4b1x5b2x6",
    ]    
)

ScroogeWalkRight02 = sprite(
    palette = { 
        "b":NES_PALETTE_HEX[0, 13],
        "w":NES_PALETTE_HEX[3, 0],
        "r":NES_PALETTE_HEX[1, 6],
        "y":NES_PALETTE_HEX[2, 8],
        "l":NES_PALETTE_HEX[3, 1],
    },
    matrix = [
        "x10b6x4",
        "x9b4w1b3x3",
        "x8b5w2b2x3",
        "x8b6w1b2x3",
        "x8b6r1b2x3",
        "x9b4r3b1x3",
        "x10b10",
        "x8b5w3b1w1b1x1",
        "x7b5w1l2w2l1w1x1",
        "x7b4w1l4w1l2x1",
        "x9b2w1l2b2w1b1l1x1",
        "x9w3l2b2w1b1l1w1",
        "x9b1w3l1b2y1b1y1w1",
        "x6b3w2b1y3b1l1y1l1x1",
        "x4b2r2b6y6",
        "x1b3r3b3w1b9",
        "b1w2b1r3b1r1b5x6",
        "b1w2b1r3b2r4b1x6",
        "x1b1w1b2r4b1r2w1b1x6",
        "x1b1w2b1r3b3r2b1x6",
        "x2b1w1b2r1b1w2b1r1b1x7",
        "x2b1w2b3w2b1r1b1x7",
        "x3b1w3b5x8",
        "x3b1y2b1y1b3x9",
        "x4b2y1b1w1x11",
        "x4b1y1b1y1b1x11",
        "x3b1y1b1y1b1x12",
        "x2b1y1b1w2b1x12",
        "x3b1w1y1w2b1x11",
        "x4b1y4b1x10",
    ]    
)

ScroogeWalkRight03 = sprite(
    palette = { 
        "b":NES_PALETTE_HEX[0, 13],
        "w":NES_PALETTE_HEX[3, 0],
        "r":NES_PALETTE_HEX[1, 6],
        "y":NES_PALETTE_HEX[2, 8],
        "l":NES_PALETTE_HEX[3, 1],
    },
    matrix = [
        "x12b6x5",
        "x11b4w1b3x4",
        "x10b5w2b2x4",
        "x10b6w1b2x4",
        "x10b6r1b2x4",
        "x11b4r3b1x4",
        "x12b2r1b7x1",
        "x10b5w3b1w1b1x2",
        "x9b5w1l2w2l1w1x2",
        "x9b4w1l4w1l2x2",
        "x11b2w1l2b2w1b1l1x2",
        "x11w3l2b2w1b1l1w1x1",
        "x11b1w3l1b2y1b1y1w1x1",
        "x9b2w2b1y3w1l1y1l1x2",
        "x7b2r1b6y7",
        "x4b3r3b3w1b9",
        "x3b1w2b1r5b3r2b1x5",
        "x3b1w2b1r6b5x5",
        "x4b1w1b1r2b1r4b2w2b1x4",
        "x4b1w2b1r2b1r3b2w2b1x4",
        "x5b1w1b2r2b2r1b2w1b1x2b2x1",
        "x5b1w3b7y1b1x1b1y2b1",
        "x3b1x2b1w7b1y1b1y1b1y3b1",
        "x2b1y1b3y2w5b1y1b1y1b1w1y2b1",
        "x2b1w1y4b1w3b2y1b2w4b1x1",
        "x1b1w2y2b2x5b1y1b2w4b1x1",
        "b1y1w2b2x6b1y1b1x2b1y1w1b1x2",
        "b1y2w2b2x5b1y1b1x2b1y2b1x2",
        "x1b1y5b1x5b1x4b2x3",
    ]    
)

DuckTalesBG01 = sprite(
    palette = {
        "b":NES_PALETTE_HEX[0, 13],
        "g":NES_PALETTE_HEX[2, 9],
        "d":NES_PALETTE_HEX[1, 10],
        "l":NES_PALETTE_HEX[1, 1],
        "r":NES_PALETTE_HEX[0, 7],
    },
    matrix = [
        "l16" + "b2r1g2r1g2r1b1r3b3",
        "l16" + "g1b1r1g1b1g1r1g1r1b1r1b1r1b3",
        "l16" + "g1b1r1g1b1g4b1r1b1r1b3",
        "l16" + "b1g1b1g1b1g3r1g1r1b1r1b3",
        "l16" + "b1g1b1r1b1g2r1b1g1r1b1r2b2",
        "l16" + "b2g1b2g2r1b1g1r1b1r2b2",
        "l16" + "b3g1b3r1b1g1r1b1r1b3",
        "l16" + "b1r1b2g2b3g1r1b1r2b2",
        "l16" + "b1r1b4g2b4r2b2",
        "l16" + "b1r1b1g1b4g3b5",
        "l16" + "b1r1b1g2b1g1b4g3b2",
        "l16" + "b1r1b1g2b1g1r1g1r1b4g1b1",
        "l8b2l5b1" + "b1r3g1b1g3r1b1r2b2g1",
        "l1b2l4b2l2b1l2b2" + "b2r1b1r1b1g2r2g1b1r1b2g1",
        "b2l2b1l1b6l1b3" + "b1r2b1g1r1g2r1b1g1b1r2b2",
        "b16" + "b16",
        "b6d1b6d1b2" + "b6d1b6d1b2",
        "b5d2b5d2b2" + "b5d2b5d2b2",
        "b1d1b2d2b2d1b2d2b3" + "b1d1b2d2b2d1b2d2b3",
        "d2b2d2b1d2b1d3b3" + "d2b2d2b1d2b1d3b3",
        "d1b2d3b1d1b2d3b2d1" + "d1b2d3b1d1b2d3b2d1",
        "d1b2d2b1d2b1d3b2d2" + "d1b2d2b1d2b1d3b2d2",
        "d1b1d3b1d2b1d3b1d3" + "d1b1d3b1d2b1d3b1d3",
        "b2d3b1d1b1d3b1d4" + "b2d3b1d1b1d3b1d4",
        "b1d4b2d4b1d4" + "b1d4b2d4b1d4",
        "d5b2d3b1d5" + "d5b2d3b1d5",
        "d5b1d10" + "d5b1d10",
        "d5b1d9b1" + "d5b1d9b1",
        "d4b2d9b1" + "d4b2d9b1",
        "b1d3b2d8b2" + "b1d3b2d8b2",
        "b7d2b2d1b2d1b1" + "b7d2b2d1b2d1b1",
        "b1g1d1b1d2b3g2b1g2b2" + "b1g1d1b1d2b3g2b1g2b2",
    ]    
)

arr01 = animate_sprites(dispmatrix=dispmatrix, 
                        sprite_list=[
                                     [ScroogeWalkRight01,
                                      ScroogeWalkRight02,
                                      ScroogeWalkRight03,
                                      ScroogeWalkRight02],
                                     ],
                        bg_sprites=[DuckTalesBG01,],
                        frame_time=0.04, 
                        center=True,
                        xoffs=[
                               [2, -1, 1, -1],
                              ],
                        yoffs=[
                               [1, 0, 1, 0],
                              ],
                        spbg_ratio=4,
                        bg_scroll_speed=(1, 0),
                        clear=True,
                        cycle_time=10,
                        transition=False,
                        transition_arr=None,
                        cycles_per_char=5, 
                        cycle_all=False)


# Under development below!

In [ ]:
ScroogeWalkRight01 = sprite(
    palette = { 
        "b":NES_PALETTE_HEX[0, 13],
        "w":NES_PALETTE_HEX[3, 0],
        "r":NES_PALETTE_HEX[1, 6],
        "y":NES_PALETTE_HEX[2, 8],
        "l":NES_PALETTE_HEX[3, 1],
    },
    matrix = [
        "x13b6x4",
        "x12b4w1b3x3",
        "x11b5w2b2x3",
        "x11b6w1b2x3",
        "x11b6r1b2x3",
        "x12b4r3b1x3",
        "x13b2r1b7",
        "x11b5w3b1w1b1x1",
        "x10b5w1l2w2l1w1x1",
        "x10b4w1l4w1l2x1",
        "x12b2w1l2b2w1b1l1x1",
        "x12w3l2b2w1b1l1w1",
        "x9b4w3l1b2y1b1y1w1",
        "x7b2r2b1w2b1y3w1l1y1l1x1",
        "x6b1r4b6y6",
        "x4b2r6b2w1b8",
        "x3b1w1b2r1b2r2b6x2b2x1",
        "x3b1w1b4r3b2r3b3w2b1",
        "x4b1w2b3r4b1w1r1b1r2w2b1",
        "x4b1w2b1w1b1r4b2r1b1r3b1x1",
        "x4b4w2b2r5b4x2",
        "x3b1y1b2w4b6x1b2x3",
        "x2b1y1b1x1b1w9b2y2b1x2",
        "x1b1y1b1x1b1x1b2w1y3b2w1b1y3b1x2",
        "b1y1b1x1b1y1b1y2b3y3b1y4b1x2",
        "y1b1x2b1y3b1x3b2y1b1w2y1b1x3",
        "b1x3b1w1b2x6b1y1w2b1x4",
        "x4b1w2b2x5b1y2b1x5",
        "x4b1y4b1x5b2x6",
    ]    
)

ScroogeWalkRight02 = sprite(
    palette = { 
        "b":NES_PALETTE_HEX[0, 13],
        "w":NES_PALETTE_HEX[3, 0],
        "r":NES_PALETTE_HEX[1, 6],
        "y":NES_PALETTE_HEX[2, 8],
        "l":NES_PALETTE_HEX[3, 1],
    },
    matrix = [
        "x10b6x4",
        "x9b4w1b3x3",
        "x8b5w2b2x3",
        "x8b6w1b2x3",
        "x8b6r1b2x3",
        "x9b4r3b1x3",
        "x10b10",
        "x8b5w3b1w1b1x1",
        "x7b5w1l2w2l1w1x1",
        "x7b4w1l4w1l2x1",
        "x9b2w1l2b2w1b1l1x1",
        "x9w3l2b2w1b1l1w1",
        "x9b1w3l1b2y1b1y1w1",
        "x6b3w2b1y3b1l1y1l1x1",
        "x4b2r2b6y6",
        "x1b3r3b3w1b9",
        "b1w2b1r3b1r1b5x6",
        "b1w2b1r3b2r4b1x6",
        "x1b1w1b2r4b1r2w1b1x6",
        "x1b1w2b1r3b3r2b1x6",
        "x2b1w1b2r1b1w2b1r1b1x7",
        "x2b1w2b3w2b1r1b1x7",
        "x3b1w3b5x8",
        "x3b1y2b1y1b3x9",
        "x4b2y1b1w1x11",
        "x4b1y1b1y1b1x11",
        "x3b1y1b1y1b1x12",
        "x2b1y1b1w2b1x12",
        "x3b1w1y1w2b1x11",
        "x4b1y4b1x10",
    ]    
)

ScroogeWalkRight03 = sprite(
    palette = { 
        "b":NES_PALETTE_HEX[0, 13],
        "w":NES_PALETTE_HEX[3, 0],
        "r":NES_PALETTE_HEX[1, 6],
        "y":NES_PALETTE_HEX[2, 8],
        "l":NES_PALETTE_HEX[3, 1],
    },
    matrix = [
        "x12b6x5",
        "x11b4w1b3x4",
        "x10b5w2b2x4",
        "x10b6w1b2x4",
        "x10b6r1b2x4",
        "x11b4r3b1x4",
        "x12b2r1b7x1",
        "x10b5w3b1w1b1x2",
        "x9b5w1l2w2l1w1x2",
        "x9b4w1l4w1l2x2",
        "x11b2w1l2b2w1b1l1x2",
        "x11w3l2b2w1b1l1w1x1",
        "x11b1w3l1b2y1b1y1w1x1",
        "x9b2w2b1y3w1l1y1l1x2",
        "x7b2r1b6y7",
        "x4b3r3b3w1b9",
        "x3b1w2b1r5b3r2b1x5",
        "x3b1w2b1r6b5x5",
        "x4b1w1b1r2b1r4b2w2b1x4",
        "x4b1w2b1r2b1r3b2w2b1x4",
        "x5b1w1b2r2b2r1b2w1b1x2b2x1",
        "x5b1w3b7y1b1x1b1y2b1",
        "x3b1x2b1w7b1y1b1y1b1y3b1",
        "x2b1y1b3y2w5b1y1b1y1b1w1y2b1",
        "x2b1w1y4b1w3b2y1b2w4b1x1",
        "x1b1w2y2b2x5b1y1b2w4b1x1",
        "b1y1w2b2x6b1y1b1x2b1y1w1b1x2",
        "b1y2w2b2x5b1y1b1x2b1y2b1x2",
        "x1b1y5b1x5b1x4b2x3",
    ]    
)

NativeWalkLeft01 = sprite(
    palette = { 
        "b":NES_PALETTE_HEX[0, 13],
        "w":NES_PALETTE_HEX[3, 0],
        "r":NES_PALETTE_HEX[1, 7],
        "y":NES_PALETTE_HEX[2, 7],
        "p":NES_PALETTE_HEX[1, 4],
    },
    matrix = [
        "x3b10x8",
        "x3b1r1b1y1b1r1b1r1b2x8",
        "x3b1r1b1y1b1r1b1r1b2x8",
        "x2b12x7",
        "x2b1r1y1r2b1r1b1r1b3x7",
        "x2b1r1y1r2b1r1b1r1b3x7",
        "x2b1r1y1r2b1r1b1r1b3x7",
        "x2b9y2b1x5b2",
        "x1b1y1b1y5w1y1b1y2b1x3b1r1b1",
        "x1b2w1y3b1w2y1b1y2b1x2b1r2b1",
        "x2b1w1b1y1b2y1p1y2b2x2b1r2b1x1",
        "b3w1b1w1b2w1y4b1x1b1r2b1x2",
        "b1y1b5y2b2y2b2r2b1x3",
        "b3y5b2y2b3r1b1x4",
        "x1b1y5b2y1b3y2b1x5",
        "x2b1y3b2y1b4y3b1x4",
        "x3b4y1b2y1b3y2r1b1x3",
        "x3b1y3b4r2b1y3b2x2",
        "x4b3x1b1r3b2r1y2b2x2",
        "x8r3b1r4b2y1b1x1",
        "x6b3r1b1r5b1y2b1x1",
        "x4w3b3r1y2r2b2y1r1b1x1",
        "x4w3b2r2y2r1b2y2b1x2",
        "x4w3b1x1b1y3b1y3b1x3",
        "x4b3x3b3y3b1x4",
        "x11b1y2b2x5",
        "x11b1y3b2x4",
        "x12b2w3b1x3",
        "x12b2w4b1x2",
        "x11b1w5b1x3",
        "x10b1w5b1x4",
    ]    
)
NativeWalkRight01 = NativeWalkLeft01.hflip()

NativeWalkLeft02 = sprite(
    palette = { 
        "b":NES_PALETTE_HEX[0, 13],
        "w":NES_PALETTE_HEX[3, 0],
        "r":NES_PALETTE_HEX[1, 7],
        "y":NES_PALETTE_HEX[2, 7],
        "p":NES_PALETTE_HEX[1, 4],
    },
    matrix = [
        "x3b10x8",
        "x3b1r1b1y1b1r1b1r1b2x8",
        "x3b1r1b1y1b1r1b1r1b2x8",
        "x2b12x7",
        "x2b1r1y1r2b1r1b1r1b3x7",
        "x2b1r1y1r2b1r1b1r1b3x4b2x1",
        "x2b1r1y1r2b1r1b1r1b3x3b1r1b1x1",
        "x2b9y2b1x2b1r2b1x1",
        "x1b1y1b1y5w1y1b1y2b2r2b1x2",
        "x1b2w1y3b1w2y1b1y2b1r2b1x3",
        "x2b1w1b1y1b2w1y3b2r2b1x4",
        "b3w1b1w1b2w1y4b1r1b1x5",
        "b1y1b5y2b2y2b2x6",
        "b3y5b2y2b4x5",
        "x1b1y5b2y1b3y3b2x3",
        "x2b1y3b2y1b5y4b1x2",
        "x3b4y1b2y1b2r1b1y2r2b1x1",
        "x3b1y3b5r2b3r2b1x1",
        "x4b5r5b4r2b1",
        "x3b1y2r2b1r3b2r1b2y3b1",
        "x3b2y2r1b5r2b2y3b1",
        "x2b1r1b4r1b1r3b2y1b2y1b1x1",
        "w3b2x3b6y4b2x1",
        "w3b9y6b1x2",
        "w3b2w3b2y6b2x1b1x1",
        "b3x1b1w4b1y1b5y2b1w1b1",
        "x5b1w3b1y1b1x4b1y2w1b1",
        "x6b1w3b1x5b3w1b1",
        "x7b1w2b1x4b1w4b1",
        "x8b2x4b1w4b1x1",
    ]    
)
NativeWalkRight02 = NativeWalkLeft02.hflip()

NativeWalkLeft03 = sprite(
    palette = { 
        "b":NES_PALETTE_HEX[0, 13],
        "w":NES_PALETTE_HEX[3, 0],
        "r":NES_PALETTE_HEX[1, 7],
        "y":NES_PALETTE_HEX[2, 7],
        "p":NES_PALETTE_HEX[1, 4],
    },
    matrix = [
        "x3b10x11",
        "x3b1r1b1y1b1r1b1r1b2x11",
        "x3b1r1b1y1b1r1b1r1b2x11",
        "x2b12x10",
        "x2b1r1y1r2b1r1b1r1b3x10",
        "x2b1r1y1r2b1r1b1r1b3x10",
        "x2b1r1y1r2b1r1b1r1b3x10",
        "x2b9y2b1x8b2",
        "x1b1y1b1y5w1y1b1y2b1x6b1r1b1",
        "x1b2w1y3b1w2y1b1y2b1x5b1r2b1",
        "x2b1w1b1y1b2y1p1y2b2x5b1r2b1x1",
        "b3w1b1w1b2w1y4b1x4b1r2b1x2",
        "b1y1b5y2b2y2b1x3b1r2b1x3",
        "b3y5b2y2b1x3b1r2b1x4",
        "x1b1y5b2y1b6r2b1x5",
        "x2b1y3b2y1b4y1r1b3x6",
        "x3b4y1b2y1r1b1y2r1b1x7",
        "x3b1y3b1r4b1y2b1r1b1x6",
        "x4b3r1b4r1y2b1r1b1x6",
        "x6b1r1b1y2b1r2y1b1r1b1x6",
        "x6b1r1b1y3r2b1r1b2x6",
        "x7b2y3r1b1r1b1y1b1x6",
        "x2b3x2b8y2b1x6",
        "x1b1w3b2y10b1x1b2x3",
        "x1b1w4b1y2b4y4b3w1b1x2",
        "x2b1w3b1y1b1x4b1y6w1b1x2",
        "x3b1w3y1b1x5b1y4w3b1x1",
        "x3b1w3b2x6b5w3b1",
        "x4b1w2b1x9b2w4b1",
        "x5b3x8b1w5b1x1",
    ]    
)
NativeWalkRight03 = NativeWalkLeft03.hflip()

DuckTalesBG01 = sprite(
    palette = {
        "b":NES_PALETTE_HEX[0, 13],
        "g":NES_PALETTE_HEX[2, 9],
        "d":NES_PALETTE_HEX[1, 10],
        "l":NES_PALETTE_HEX[1, 1],
        "r":NES_PALETTE_HEX[0, 7],
    },
    matrix = [
        "l16" + "b2r1g2r1g2r1b1r3b3",
        "l16" + "g1b1r1g1b1g1r1g1r1b1r1b1r1b3",
        "l16" + "g1b1r1g1b1g4b1r1b1r1b3",
        "l16" + "b1g1b1g1b1g3r1g1r1b1r1b3",
        "l16" + "b1g1b1r1b1g2r1b1g1r1b1r2b2",
        "l16" + "b2g1b2g2r1b1g1r1b1r2b2",
        "l16" + "b3g1b3r1b1g1r1b1r1b3",
        "l16" + "b1r1b2g2b3g1r1b1r2b2",
        "l16" + "b1r1b4g2b4r2b2",
        "l16" + "b1r1b1g1b4g3b5",
        "l16" + "b1r1b1g2b1g1b4g3b2",
        "l16" + "b1r1b1g2b1g1r1g1r1b4g1b1",
        "l8b2l5b1" + "b1r3g1b1g3r1b1r2b2g1",
        "l1b2l4b2l2b1l2b2" + "b2r1b1r1b1g2r2g1b1r1b2g1",
        "b2l2b1l1b6l1b3" + "b1r2b1g1r1g2r1b1g1b1r2b2",
        "b16" + "b16",
        "b6d1b6d1b2" + "b6d1b6d1b2",
        "b5d2b5d2b2" + "b5d2b5d2b2",
        "b1d1b2d2b2d1b2d2b3" + "b1d1b2d2b2d1b2d2b3",
        "d2b2d2b1d2b1d3b3" + "d2b2d2b1d2b1d3b3",
        "d1b2d3b1d1b2d3b2d1" + "d1b2d3b1d1b2d3b2d1",
        "d1b2d2b1d2b1d3b2d2" + "d1b2d2b1d2b1d3b2d2",
        "d1b1d3b1d2b1d3b1d3" + "d1b1d3b1d2b1d3b1d3",
        "b2d3b1d1b1d3b1d4" + "b2d3b1d1b1d3b1d4",
        "b1d4b2d4b1d4" + "b1d4b2d4b1d4",
        "d5b2d3b1d5" + "d5b2d3b1d5",
        "d5b1d10" + "d5b1d10",
        "d5b1d9b1" + "d5b1d9b1",
        "d4b2d9b1" + "d4b2d9b1",
        "b1d3b2d8b2" + "b1d3b2d8b2",
        "b7d2b2d1b2d1b1" + "b7d2b2d1b2d1b1",
        "b1g1d1b1d2b3g2b1g2b2" + "b1g1d1b1d2b3g2b1g2b2",
    ]    
)

DuckTalesBG02 = sprite(
    palette = {
        "b":NES_PALETTE_HEX[0, 13],
        "g":NES_PALETTE_HEX[1, 10],
        "c":NES_PALETTE_HEX[0, 12],
        "l":NES_PALETTE_HEX[2, 1],
        "u":NES_PALETTE_HEX[1, 1],
    },
    matrix = [
        "c1b1u1c3b1l1b7l2u3b1u1b4u1c3b1c1",
        "c1b1u1c3b1u1b6c1l2u2b1l1b5u1c3b1c1",
        "c1b1u1c3b8c1u1l1u1b1l1b1c1b4u1c3b1c1",
        "c1b2u1c3b6c1b1u2b1l1b1c2b3u1c3b1c2",
        "c2b1u1c3b6u1b3l1b1c2b4u1c3b1c2",
        "c2b1u1c4b5l1b1l2b1c2b4u1c4b1c2",
        "c2b2u1c3b5u1l1u1b8u1c3b1c3",
        "c3b1c4u1b14u1c4b1c3",
        "c4b1c2l1b1u1b12u1l1b1c2b1c4",
        "c5b1c1b2c1u2b8u2c1b2c1b1c5",
        "c6b1c5u8c5b1c6",
        "c7b1c7l1b1c7b1c7",
        "c8b2c5b2c5b2c8",
        "c10b3c6b3c10",
        "c13b6c13",
        
        "c32",
        "c32",
        "c32",
        "c32",
        "c32",
        "c32",
        "c32",
        "c32",
        "c32",
        "c32",
        "c32",
        "c32",
        "c32",
        "c32",
        "c32",
        "c32",       
        "g7b1g24"
    ]    
)

arr01 = animate_sprites(dispmatrix=dispmatrix, 
                        sprite_list=[
                                     [ScroogeWalkRight01,
                                      ScroogeWalkRight02,
                                      ScroogeWalkRight03,
                                      ScroogeWalkRight02],
                                     [NativeWalkRight02,
                                      NativeWalkRight01,
                                      NativeWalkRight03,
                                      NativeWalkRight01,]
                                     ],
                        bg_sprites=[DuckTalesBG02,],
                        frame_time=0.04, 
                        center=True,
                        xoffs=[
                               [2, -1, 1, -1],
                               [0, 0, 1, 0],
                              ],
                        yoffs=[
                               [1, 0, 1, 0],
                               [0, 0, 0, 0],
                              ],
                        spbg_ratio=4,
                        bg_scroll_speed=(1, 0),
                        clear=True,
                        cycles=None,
                        transition=False,
                        transition_arr=None,
                        cycles_per_char=5, 
                        cycle_all=False)



In [ ]:
#Test animation

temp = excitebike_animation
arr01 = animate_sprites(dispmatrix=dispmatrix, 
                        sprite_list=temp.sprite_list,
                        bg_sprites=temp.bg_sprites,                                  
                        frame_time=temp.frame_time, 
                        center=temp.center,
                        xoffs=temp.xoffs,
                        yoffs=temp.yoffs,
                        spbg_ratio=temp.spbg_ratio,
                        bg_scroll_speed=temp.bg_scroll_speed,
                        clear=True,
                        cycles=None,
                        transition=False,
                        transition_arr=None,
                        cycles_per_char=5, 
                        cycle_all=False)